# 🏢 Company Intelligence Agent - Interactive Demo

This notebook demonstrates the Multi-Agent System for generating company intelligence reports using LangGraph.

## Overview

The system uses two collaborating AI agents:
1. **Data Collector Agent**: Fetches news and stock data
2. **Analyst Agent**: Generates summaries and risk assessments

Let's explore each component step by step!

## 1. Setup & Imports

In [ ]:
# Install dependencies (if needed)
# !pip install -q langgraph langchain langchain-openai yfinance python-dotenv

In [ ]:
import sys
import os

# Add parent directory to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✅ Setup complete!")
print(f"OpenAI API Key configured: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No (using simulated data)'}")

## 2. Understanding the Tools

The system uses four specialized tools. Let's test each one individually.

### 2.1 News Fetcher Tool

In [ ]:
from src.tools.news_fetcher import news_fetcher_tool

# Fetch news for a company
company = "Apple"
news = news_fetcher_tool.invoke({"company_name": company})

print(f"📰 News articles for {company}:\n")
for i, article in enumerate(news, 1):
    sentiment = "🟢" if article['sentiment_score'] > 0.2 else "🔴" if article['sentiment_score'] < -0.2 else "🟡"
    print(f"{i}. {sentiment} {article['headline']}")
    print(f"   Source: {article['source']} | Category: {article['category']}\n")

### 2.2 Stock Data Tool

In [ ]:
from src.tools.stock_data import stock_data_tool

# Fetch stock data for a company
stock = stock_data_tool.invoke({"company_name": company})

print(f"📈 Stock Data for {company}:\n")
print(f"Ticker: {stock.get('ticker', 'N/A')}")
print(f"Current Price: ${stock.get('current_price', 'N/A')}")
print(f"Daily Change: {stock.get('daily_change_percent', 'N/A'):+.2f}%")
print(f"Monthly Change: {stock.get('monthly_change_percent', 'N/A'):+.2f}%")
print(f"Market Cap: ${stock.get('market_cap', 0):,}")
print(f"P/E Ratio: {stock.get('pe_ratio', 'N/A')}")
print(f"Data Source: {stock.get('data_source', 'N/A')}")

### 2.3 Summary Generator Tool

In [ ]:
from src.tools.summary_generator import summary_generator_tool, format_news_for_summary, format_stock_for_summary

# Format the data
news_summary = format_news_for_summary(news)
stock_summary = format_stock_for_summary(stock)

# Generate summary
summary = summary_generator_tool.invoke({
    "company_name": company,
    "news_summary": news_summary,
    "stock_summary": stock_summary
})

print(summary)

### 2.4 Risk Analyzer Tool

In [ ]:
from src.tools.risk_analyzer import risk_analyzer_tool

# Analyze risks
risk_result = risk_analyzer_tool.invoke({
    "company_name": company,
    "news_data": news,
    "stock_data": stock
})

print(f"⚠️ Risk Assessment for {company}:\n")
print(f"Risk Level: {risk_result.get('risk_level', 'UNKNOWN')}")
print(f"Description: {risk_result.get('risk_description', 'N/A')}\n")

print("Risk Factors:")
for risk in risk_result.get('all_risks', []):
    print(f"  • {risk}")

## 3. Understanding the Agents

Now let's see how the agents work together.

### 3.1 Data Collector Agent

In [ ]:
from src.agents.data_collector import collect_data_directly

# Run the Data Collector agent
data_result = collect_data_directly("Microsoft")

print(f"📊 Data Collector Results for Microsoft:\n")
print(f"News articles collected: {len(data_result.get('news_data', []))}")
print(f"Stock data retrieved: {'Yes' if data_result.get('stock_data') else 'No'}")
print(f"Current agent: {data_result.get('current_agent')}")

### 3.2 Analyst Agent

In [ ]:
from src.agents.analyst import analyze_data_directly

# Run the Analyst agent
analysis_result = analyze_data_directly(
    "Microsoft",
    data_result.get('news_data', []),
    data_result.get('stock_data', {})
)

print(f"🔍 Analyst Results for Microsoft:\n")
print(f"Risk Level: {analysis_result.get('risk_level', 'N/A')}")
print(f"Risk Factors: {len(analysis_result.get('risk_factors', []))}")
print(f"\nAnalysis Preview:")
print(analysis_result.get('analysis', 'N/A')[:500] + "...")

## 4. The Orchestrated Workflow

Now let's run the complete orchestrated workflow using LangGraph.

In [ ]:
from src.orchestrator.supervisor import run_intelligence_workflow, get_workflow_graph_visualization

# Visualize the workflow
print(get_workflow_graph_visualization())

In [ ]:
# Run the complete workflow
result = run_intelligence_workflow("Tesla", thread_id="notebook-demo")

# Display the final report
print(result.get('final_report', 'No report generated'))

## 5. Understanding the Shared State

The shared state maintains context across all agent interactions.

In [ ]:
from src.state.shared_state import AgentState, create_initial_state

# Create an initial state
state = create_initial_state("Google")

print("📋 Initial State Structure:")
for key, value in state.items():
    print(f"  • {key}: {type(value).__name__} = {value if value is not None else 'None'}")

## 6. Try It Yourself!

Enter a company name below to generate an intelligence report.

In [ ]:
# Change this to any company you want to analyze
YOUR_COMPANY = "Amazon"

# Run the analysis
your_result = run_intelligence_workflow(YOUR_COMPANY, thread_id="your-analysis")

# Display the report
print(your_result.get('final_report', 'No report generated'))

## 7. Key Takeaways

### What We Learned:

1. **Multi-Agent Architecture**: Two specialized agents (Data Collector & Analyst) work together
2. **Tool-Based Capabilities**: Each agent has access to specific tools for its role
3. **LangGraph Orchestration**: StateGraph manages workflow with conditional routing
4. **Shared State**: Context is maintained across all agent interactions
5. **Memory Persistence**: Sessions can be tracked using thread IDs

### Next Steps:

- Try the Streamlit UI: `streamlit run ui/streamlit_app.py`
- Customize the agents' prompts and tools
- Add new tools for additional data sources
- Extend the workflow with more agents